In [55]:
pip install tensorflow librosa numpy matplotlib jiwer

In [56]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

In [57]:
def process_audio(audio_path):
    y,sr = librosa.load(audio_path,sr=16000)
    mel_spec = librosa.feature.melspectrogram(y=y,sr=sr,n_mels=128)
    mel_spec_db = librosa.power_to_db(mel_spec,ref=np.max)

    plt.figure(figsize=(10,4))
    librosa.display.specshow(mel_spec_db,sr=sr,x_axis="time",y_axis="mel")
    plt.colorbar(format="%+2.0f dB")
    plt.title("Mel Spectrogram")
    plt.show()

    return mel_spec


In [58]:
# process_audio("/content/ttsMP3.com_VoiceText_2025-3-11_12-8-46.wav")

In [59]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [60]:
def transcribe(audio_path):
    y,sr = librosa.load(audio_path,sr=16000)
    input_values = processor(y,return_tensors='pt',sampling_rate=sr).input_values

    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits,dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]
    return transcription

In [61]:
# print(transcribe('/content/ttsMP3.com_VoiceText_2025-3-11_12-8-46.wav'))

In [ ]:
# from transformers import TrainingArguments, Trainer

# training_args = TrainingArguments(
#     output_dir="./wav2vec2_model",
#     per_device_train_batch_size=8,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=3e-5,
#     num_train_epochs=3,
#     logging_dir="./logs"
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,  # Your custom dataset
#     eval_dataset=eval_dataset
# )

# trainer.train()

C:\Users\Priya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`

In [ ]:
print(transcribe('Rec_wav.wav'))

THE DAMABOR WAS A SONING I SPEED ROGET
